In [63]:
import numpy as np
import operator
import feedparser

In [3]:
def loadDataSet():
    postingList=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0,1,0,1,0,1]    #1 is abusive, 0 not
    
    return postingList,classVec

def createVocabList(dataSet):
    vocabSet = set([])
    
    for document in dataSet:
        vocabSet = vocabSet | set(document)
    
    return list(vocabSet)

def setOfWords2Vec(vocabList , inputSet):
    returnVec = [0]*len(vocabList)
    
    for word in inputSet:
        if word in inputSet:
            returnVec[vocabList.index(word)] = 1
        else:
            print('%s not in my va' % word)
    
    return returnVec


In [4]:
listOPosts , listClasses = loadDataSet()
myVocabList = createVocabList(listOPosts)

In [5]:
myVocabList

['is',
 'not',
 'stop',
 'how',
 'cute',
 'please',
 'has',
 'garbage',
 'mr',
 'dog',
 'my',
 'flea',
 'so',
 'love',
 'stupid',
 'take',
 'quit',
 'problems',
 'buying',
 'I',
 'posting',
 'maybe',
 'ate',
 'him',
 'to',
 'worthless',
 'licks',
 'steak',
 'food',
 'dalmation',
 'help',
 'park']

In [6]:
setOfWords2Vec(myVocabList , listOPosts[0])

[0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0]

In [38]:
def trainNB0(trainMatrix , trainCategory):
    numTrainDocs = len(trainMatrix) #训练样本数量
    numWords = len(trainMatrix[0]) #样本的词向量长度
    pAbusive = np.sum(trainCategory)/float(numTrainDocs) #计算p(侵辱类的 因为label=1 所以直接相加 非侵辱的就1-p)
    
    #统计词向量每一个词出现的频率 （given label） 条件概率
    #分子
    #拉普拉斯平滑
    p0Num = np.ones(numWords)#p0Num = np.zeros(numWords)
    p1Num = np.ones(numWords)#p1Num = np.zeros(numWords)
    #分母
    #p0Demon = 0.0
    #p1Demon = 0.0
    #拉普拉斯平滑
    p0Demon = 2.0
    p1Demon = 2.0
    
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]
            p1Demon += np.sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Demon += np.sum(trainMatrix[i])
    
    #p1Vect = p1Num / p1Demon
    #p0Vect = p0Num / p0Demon
    
    #防止后续相乘 出现过小 下溢出
    p1Vect = np.log(p1Num / p1Demon)
    p0Vect = np.log(p0Num / p0Demon)
    
    #print(p1Demon , p0Demon)
    
    
    return p0Vect , p1Vect , pAbusive

In [11]:
trainMat = []

for postinDoc in listOPosts:
    trainMat.append(setOfWords2Vec(myVocabList , postinDoc))

In [32]:
p0V , p1V , pAb = trainNB0(trainMat , listClasses)

In [42]:
type(listClasses[0])

int

In [34]:
p0V

array([-2.56494936, -3.25809654, -2.56494936, -2.56494936, -2.56494936,
       -2.56494936, -2.56494936, -3.25809654, -2.56494936, -2.56494936,
       -1.87180218, -2.56494936, -2.56494936, -2.56494936, -3.25809654,
       -3.25809654, -3.25809654, -2.56494936, -3.25809654, -2.56494936,
       -3.25809654, -3.25809654, -2.56494936, -2.15948425, -2.56494936,
       -3.25809654, -2.56494936, -2.56494936, -3.25809654, -2.56494936,
       -2.56494936, -3.25809654])

In [35]:
p1V

array([-3.04452244, -2.35137526, -2.35137526, -3.04452244, -3.04452244,
       -3.04452244, -3.04452244, -2.35137526, -3.04452244, -1.94591015,
       -3.04452244, -3.04452244, -3.04452244, -3.04452244, -1.65822808,
       -2.35137526, -2.35137526, -3.04452244, -2.35137526, -3.04452244,
       -2.35137526, -2.35137526, -3.04452244, -2.35137526, -2.35137526,
       -1.94591015, -3.04452244, -3.04452244, -2.35137526, -3.04452244,
       -3.04452244, -2.35137526])

In [43]:
def classifyNB(vec2Classify , p0Vec , p1Vec , pClass1):
    p1 = np.sum(vec2Classify * p1Vec) + np.log(pClass1)
    p0 = np.sum(vec2Classify * p0Vec) + np.log(1-pClass1)
    
    if p1>p0:
        return 1
    else:
        return 0

def testingNB():
    listOPosts , listClasses= loadDataSet()
    myVocabList = createVocabList(listOPosts)
    trainMat = []
    
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList , postinDoc))
    
    p0V , p1V , pAb = trainNB0(np.array(trainMat) , np.array(listClasses))
    testEntry = ['love' , 'my' , 'dalmation']
    thisDoc = np.array(setOfWords2Vec(myVocabList , testEntry))
    
    print('class result:',classifyNB(thisDoc , p0V , p1V , pAb))
    
    testEntry = ['stupid' , 'garbage']
    thisDoc = np.array(setOfWords2Vec(myVocabList , testEntry))
    
    print('class result:',classifyNB(thisDoc , p0V , p1V , pAb))

In [44]:
testingNB()

class result: 0
class result: 1


In [77]:
#每一个词 是否出现作为一个特征 ：词集模型
#每一个词 在文档出现不止一次 出现了多次 可能包含所不能表达的某种信息 ：词袋模型
def bagOfWords2Vec(vocabList , inputSet):
    returnVec = [0]*len(vocabList)
    
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1
        else:
            print('%s not in my va' % word)
    
    return returnVec

In [46]:
#切分文本 使用正则表达式处理
import re

In [47]:
mySent = 'This is the best Python.Never upon.'
mySent.split()

['This', 'is', 'the', 'best', 'Python.Never', 'upon.']

In [49]:
regEx = re.compile('\\W*') #分隔符是除了 单词 数字以外的任意字符串
listsTokens = regEx.split(mySent)
listsTokens

I:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: split() requires a non-empty pattern match.
  


['This', 'is', 'the', 'best', 'Python', 'Never', 'upon', '']

In [50]:
#上面结果有空字符串 将之去除
[tok for tok in listsTokens if len(tok) > 0]

['This', 'is', 'the', 'best', 'Python', 'Never', 'upon']

In [51]:
#大小写用不上 全部改为小写
[tok.lower() for tok in listsTokens if len(tok) > 0]

['this', 'is', 'the', 'best', 'python', 'never', 'upon']

In [60]:
#一个大的字符串分解函数 分解为一些小的单词构成的
#也是一种解析操作
def textParse(bigString):
    listsTokens = re.split(r'\W*' , bigString) #''前面的r 不进行转义处理
    return [tok.lower for tok in listsTokens if len(tok)>2 ] #将长度小于3的字符串全部过滤掉


def spamTest():
    docList = []
    classList = []
    fullList = []
    
    for i in range(1,26):
        wordList = textParse(open('../MLiA_SourceCode/Ch04/email/spam/%d.txt' % i).read())
        
        docList.append(wordList)
        fullList.append(wordList)
        classList.append(1)
        
        wordList = textParse(open('../MLiA_SourceCode/Ch04/email/spam/%d.txt' % i).read())
        
        docList.append(wordList)
        fullList.append(wordList)
        classList.append(0)
    
    vocabList = createVocabList(docList)
    
    trainingSet = list(range(50))
    
    #测试样本
    testSet = []
    
    #trainingSet 只是一个预设的索引值
    for i in range(10):
        randIndex = int(np.random.uniform(0 , len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del(trainingSet[randIndex])
    
    #训练样本
    trainMat = []
    trainClasses = []
    
    #现在trainingSet留下的是进行训练的样本
    for docIndex in trainingSet:
        trainMat.append(setOfWords2Vec(vocabList , docList[docIndex]))
        trainClasses.append(classList[docIndex])
    
    #训练
    p0V , p1V , pSpam = trainNB0(np.array(trainMat) , np.array(trainClasses))
    errorCount = 0
    
    for docIndex in testSet:
        wordVector = setOfWords2Vec(vocabList , docList[docIndex])
        if classifyNB(np.array(wordVector) , p0V , p1V , pAb) != classList[docIndex]:
            errorCount += 1
    
    print('error rate:%f' % (float(errorCount) / len(testSet)))

In [62]:
spamTest()

error rate:0.400000


I:\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [57]:
list(range(1,4))

[1, 2, 3]

In [64]:
ny = feedparser.parse('http://newyork.craigslist.org/stp/index.rss')

In [65]:
ny['entries']

[{'dc_source': 'https://newyork.craigslist.org/mnh/stp/d/hey-are-you-420-friendly-m4w/6406622811.html',
  'dc_type': 'text',
  'id': 'https://newyork.craigslist.org/mnh/stp/d/hey-are-you-420-friendly-m4w/6406622811.html',
  'language': 'en-us',
  'link': 'https://newyork.craigslist.org/mnh/stp/d/hey-are-you-420-friendly-m4w/6406622811.html',
  'links': [{'href': 'https://newyork.craigslist.org/mnh/stp/d/hey-are-you-420-friendly-m4w/6406622811.html',
    'rel': 'alternate',
    'type': 'text/html'}],
  'published': '2017-11-30T08:49:36-05:00',
  'published_parsed': time.struct_time(tm_year=2017, tm_mon=11, tm_mday=30, tm_hour=13, tm_min=49, tm_sec=36, tm_wday=3, tm_yday=334, tm_isdst=0),
  'rights': 'copyright 2017 craiglist',
  'rights_detail': {'base': 'https://newyork.craigslist.org/search/stp?format=rss',
   'language': None,
   'type': 'text/plain',
   'value': 'copyright 2017 craiglist'},
  'summary': "Hi Craigslist I'm looking for a 420 buddy preferrely a woman. I'm a 25 year old

In [75]:
#下面使用贝叶斯 观察单词 概率 特定城市之间的关系

#统计出现的次数
def calcMostFrep(vocabList , fullText):
    freqDict = {}
    
    for token in vocabList:
        freqDict[token] = fullText.count(token)
    
    sortedFreq = sorted(freqDict.items() , key = operator.itemgetter(1) , reverse=True)
    
    return sortedFreq[:30]

def localWords(feed1 , feed0):
    docList = []
    classList = []
    fullText = []
    
    minLen = min(len(feed1['entries']) , len(feed0['entries']))
    
    for i in range(minLen):
        wordList = textParse(feed1['entries'][i]['summary'])
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1)
        
        wordList = textParse(feed0['entries'][i]['summary'])
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    
    vocabList = createVocabList(docList)
    top30Words = calcMostFrep(vocabList , fullText)
    
    for pairW in top30Words:
        if pairW[0] in vocabList:
            vocabList.remove(pairW[0])
    
    trainingSet = list(range(2*minLen))
    testSet = []
    
    for i in range(20):
        randIndex = int(np.random.uniform(0 , len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del(trainingSet[randIndex])
    
    trainMat = []
    trainClasses = []
    
    for docIndex in trainingSet:
        trainMat.append(bagOfWords2Vec(vocabList , docList[docIndex]))
        trainClasses.append(classList[docIndex])
    
    p0V , p1V , pAb = trainNB0(np.array(trainMat) , np.array(trainClasses))
    
    errorCount = 0
    
    for docIndex in testSet:
        wordVector = bagOfWords2Vec(vocabList , docList[docIndex])
        if classifyNB(np.array(wordVector) , p0V , p1V , pAb) != classList[docIndex]:
            errorCount += 1
        
    print('error rate:%f' % (float(errorCount) / len(testSet)))
    
    return vocabList , p0V , p1V

In [81]:
ny = feedparser.parse('http://newyork.craigslist.org/stp/index.rss')
sf = feedparser.parse('http://sfbay.craigslist.org/stp/index.rss')


In [82]:
vocabList , pSF , pNY = localWords(ny , sf)

<built-in method lower of str object at 0x0000027CF89C0688> not in my va
<built-in method lower of str object at 0x0000027CF889E3B0> not in my va
<built-in method lower of str object at 0x0000027CF89F6A08> not in my va
<built-in method lower of str object at 0x0000027CF88B1AB0> not in my va
<built-in method lower of str object at 0x0000027CF8F47110> not in my va
<built-in method lower of str object at 0x0000027CF8F47148> not in my va
<built-in method lower of str object at 0x0000027CF8880AE8> not in my va
<built-in method lower of str object at 0x0000027CF96B13B0> not in my va
<built-in method lower of str object at 0x0000027CF96B14B0> not in my va
<built-in method lower of str object at 0x0000027CF9691730> not in my va
<built-in method lower of str object at 0x0000027CF9691810> not in my va
<built-in method lower of str object at 0x0000027CF96B1630> not in my va
<built-in method lower of str object at 0x0000027CF96B1730> not in my va
<built-in method lower of str object at 0x0000027CF

I:\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [88]:
def getTopWords(ny , sf):
    vocaList , p0V , p1V = localWords(ny , sf)
    topNY = []
    topSF = []
    
    for i in range(len(p0V)):
        if p0V[i] > -6.0:
            topSF.append((vocabList[i] , p0V[i]))
            
        if p1V[i] > -6.0:
            topNY.append((vocabList[i] , p1V[i]))
        
    sortedSF = sorted(topSF , key = lambda pair :pair[1] , reverse=True)
    print('===')
    for item in sortedSF:
        print(item[0])
    print('===')
    sortedNY = sorted(topNY , key = lambda pair :pair[1] , reverse=True)
    for item in sortedNY:
        print(item[0])
    

In [89]:
getTopWords(ny , sf)

I:\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


<built-in method lower of str object at 0x0000027CF96E8230> not in my va
<built-in method lower of str object at 0x0000027CF96E8070> not in my va
<built-in method lower of str object at 0x0000027CF7D92420> not in my va
<built-in method lower of str object at 0x0000027CF7D920D8> not in my va
<built-in method lower of str object at 0x0000027CF7D47C70> not in my va
<built-in method lower of str object at 0x0000027CF8E96228> not in my va
<built-in method lower of str object at 0x0000027CF8E96810> not in my va
<built-in method lower of str object at 0x0000027CF8E96030> not in my va
<built-in method lower of str object at 0x0000027CF8C027A0> not in my va
<built-in method lower of str object at 0x0000027CF8C02848> not in my va
<built-in method lower of str object at 0x0000027CF89C7458> not in my va
<built-in method lower of str object at 0x0000027CF55C6DF0> not in my va
<built-in method lower of str object at 0x0000027CF8FCE2B0> not in my va
<built-in method lower of str object at 0x0000027CF

<built-in method lower of str object at 0x0000027CF9695880>
<built-in method lower of str object at 0x0000027CF9690618>
<built-in method lower of str object at 0x0000027CF8880EA0>
<built-in method lower of str object at 0x0000027CF8F3E810>
<built-in method lower of str object at 0x0000027CF9695F10>
<built-in method lower of str object at 0x0000027CF96CCF48>
<built-in method lower of str object at 0x0000027CF8880D18>
<built-in method lower of str object at 0x0000027CF88BB5A8>
<built-in method lower of str object at 0x0000027CF88B1068>
<built-in method lower of str object at 0x0000027CF889F6C0>
<built-in method lower of str object at 0x0000027CF89D9EA0>
<built-in method lower of str object at 0x0000027CF889FB90>
<built-in method lower of str object at 0x0000027CF89F6340>
<built-in method lower of str object at 0x0000027CF8F47228>
<built-in method lower of str object at 0x0000027CF88B1F48>
<built-in method lower of str object at 0x0000027CF889FB20>
<built-in method lower of str object at 